## 러스트에서 제너릭은 

- 코드 재사용성과 추상화를 위해 사용되는 매우 강력한 기능입니다. 러스트에서는 크게 두 가지 종류의 제너릭이 있습니다.

### 함수 제너릭 (Function generics)
- 함수 제너릭은 함수 내에서 인자의 타입이나 반환값의 타입을 추론할 수 있게 하는 기능입니다. 
- 함수 제너릭을 사용하여 여러 타입의 인자와 반환값을 처리할 수 있습니다. 

### 타입 제너릭 (Type generics)

- 타입 제너릭은 구조체나 열거형 등 타입 자체에 대한 제너릭입니다. 
- 타입 제너릭을 사용하여 여러 다른 타입을 포함하는 컨테이너 타입을 작성할 수 있습니다. 


## 1. 제너릭 함수

- 타입을 타입매개변수로 지정해서 여러 타입을 하나의 함수에서 처리가 가능
- 다양한 타입의 일반화된 함수를 제공 

## 1-1 타입 정의 제너릭 함수  

- 일반함수를 작성할 때는 매개변수와 반환값에 대한 타입을 지정한다. 

### 타입 출력 함수 

In [2]:
fn print_type<T>(value: T) {
    println!("value has type: {}", std::any::type_name::<T>());
}

In [3]:
print_type(100);

value has type: i32


### 타입확인한 결과를 문자열로 출력  제너릭 함수 정의 

In [170]:

// Get the type of given variable, return a string representation of the type  , e.g "i8", "u8", "i32", "u32"
fn type_of<T>(_: &T) -> String {
    format!("{}", std::any::type_name::<T>())
}

## 1-2. 일반제너릭 정의 

- 제너릭으로 지정한 타입은 연산자를 인식할 수 없다.
- 제너릭에 특정 연산자를 사용하려면 트레이트로 제한을 지정해야 연산자를 사용할 수 있다

### 덧셈을 처리하는 일반함수 

In [2]:
fn add(x:i32,y:i32) -> i32 {
    x + y
}

In [3]:
add(100,200)

300

### 두 매개변수를 더하는 제너릭 함수 정의

- 타입매개변수는 함수명 다름에 꺾쇠 기호 다음에 넣고 
- 이 타입매개변수를 매개변수의 타입으로 지정한다, 또한 반환값도 지정할 수 있다.
- 하지만 이 타입은 명확히 연산자를 사용할 수 있는지 알 수가 없다. 
- 그래서 연산자 처리에 대해서 예외를 발생시킨다, 

In [4]:
fn add_gen<T> (x:T, y:T) -> T {
    x+y
}

Error: cannot add `T` to `T`

## 1-3 트레이트 제한

- trait bound : 트레이트 제한이나 트레이트 경계
- 타입 매개변수에 특정한 기능을 제공하는 범위를 트레이트로 지정해서 확정한다. 

### 바운드 
- 제네릭 타입이 가져야 하는 특정 속성을 정의하는 것으로, 
- 제네릭 타입을 사용할 때 타입의 범위를 제한하고 안정성을 높일 수 있습니다.

In [4]:
use std::fmt::Display;

fn print_type<T: Display>(value: T) {
    println!("value is {}", value);
}

In [5]:
print_type(100);

value is 100


### 여러 개의 바운드 사용 

-  덧셈 기호로 트레이트를 연결한다

In [6]:
fn print_type_1<T: Display + Clone>(value: T) {
    println!("value is {}", value);
}

In [7]:
print_type_1("문자열".to_owned());

value is 문자열


## 1-4 트레이트 제한 : 연산자 

### 덧셈 트레이트를 사용한다

- 트레이트를 지정할 때는 현재 작성된 함수에 트레이트 정보가 있어야 한다

In [5]:
use std::ops::Add

### Add 트레이트
- 연관타입을 가짐 
- 그래서 연관타입 정보도 제공한다. 

In [9]:
fn add_gen<T:Add<Output=T>>(x:T, y:T) -> T {
    x+y
}

In [15]:
add_gen(10,20)

30

In [16]:
add_gen(10.2, 20.3)

30.5

### 문자열을 덧셈처리 안됨 

In [18]:
add_gen("abc".to_string(), "cde".to_string())

Error: cannot add `String` to `String`

### 문자열간 결합을 쉽게 하기 위해서 format 매크로를 사용한다

In [39]:
fn add_sgen(_x:String, _y:String) -> String {
    format!("{}{}",_x,_y)
    
}

In [40]:
add_sgen("가을".to_string(),"겨울".to_string())

"가을겨울"

### 위의 방식중에 매크로 연결해서 처리

- 매크로에 문자열리터럴을 넣고 연결해서 사용이 가능하다. 

In [38]:
concat!("가을","겨울")

"가을겨울"

## 1-4 다양한 트레이트 제한을 표시하기 : where

- 여러 개의 트레이트를 제한하기 위해서는 where 표현식으로 트레이트를 제한해야 함

- 반환자료형 다음에 where를 사용해서 트레이트 제한을 처리한다. 특히 트레이트 제한이 많은 경우 사용한다 


### 제너릭 함수에 바로 where 를 사용

In [62]:
fn add_str(x: &str) ->String {
    x.to_string()
}

In [63]:
add_str("겨울이")

"겨울이"

In [160]:
// 'static as part of a trait bound:

fn generic<T>(x: T) where T: std::fmt::Debug {
    println!("{:?}",x);
}

In [161]:
generic("여름이")

"여름이"


()

In [162]:
fn add_gen2<T>(x:T, y:T) -> String where T:std::fmt::Display {
    format!("{}{}", x, y)
}

In [163]:
"가을이".to_string() + "겨울이"

"가을이겨울이"

In [164]:
add_gen2("가을이","겨울이")

"가을이겨울이"

### 구조체와 트레이트를 사용한 경우 이를 트레이트 경계로 사용 

In [2]:
struct SeaCreature {
    pub name: String,
    noise: String,
}

impl SeaCreature {
    pub fn get_sound(&self) -> &str {
        &self.noise
    }
}

trait NoiseMaker {
    fn make_noise(&self);
}

impl NoiseMaker for SeaCreature {
    fn make_noise(&self) {
        println!("{}", &self.get_sound());
    }
}

fn generic_make_noise<T>(creature: &T)
where
    T: NoiseMaker,
{
    // 컴파일 타임에 실제 자료형을 알게 됩니다
    creature.make_noise();
}

fn main() {
    let creature = SeaCreature {
        name: String::from("Ferris"),
        noise: String::from("blub"),
    };
    generic_make_noise(&creature);
}


In [3]:
main();

blub


### 벡터를 출력하기 

In [9]:
fn print_vector<T>(vector: &Vec<T>) 
where
    T: std::fmt::Debug,
{
    for element in vector {
        println!("{:?}", element);
    }
}

fn main() {
    let v1 = vec![1, 2, 3];
    let v2 = vec!["a", "b", "c"];

    print_vector(&v1);
    print_vector(&v2);
}

In [10]:
main();

1
2
3
"a"
"b"
"c"


## 1-5 숫자 구하기  제너릭

In [219]:
use std::any::{Any, TypeId};

In [246]:
fn addition_of_values<T: PartialOrd + std::ops::Add<Output = T> + Into<T> + From<T> >(num1: T, num2: T) -> T {
    num1.into()  + num2
   

}

In [247]:
addition_of_values(100,200)

300

In [248]:
addition_of_values(100.22,200.33)

300.55

In [252]:
addition_of_values("가을이", "겨율이")

Error: cannot add `&str` to `&str`

In [196]:
Into::<String>::into("aaa")

"aaa"

In [181]:
type_of(&"str".to_string())

"alloc::string::String"

In [182]:
"str".to_string().type_name_of()

"std::string::String"

## 2. 문자열 연결 처리를 알아보기 

### 더하기 연산자로 문자열을 연결하기

- 더하기 연산으로 문자열을 연결할 때 첫번째는 문자열이고 두번째는 문자열 리터럴이다.  

In [90]:
fn main() {
    let one = "string";
    let two = "something else";

    let three = one.to_owned() + two;   // to_owned는 소유권 데이터로 변환 -> 복제됨 
    let four = one.to_string();
    
    println!(" {} ", three);
    println!(" {} ", one);
    println!(" {} ", four);
}

In [91]:
main()

 stringsomething else 
 string 
 string 


()

###  두 개의 문자리터럴 연결하기 : join 메서드 

- 두 개의 문자열 리터럴을 받고 배열에 넣고 join 메서드로 처리하면 문자열로 반환한다.


In [97]:
fn add_join(first:&str, second:&str) -> String {
    [first, second].join(" ")
}


In [98]:
add_join("자자","오오")

"자자 오오"

###  두 개의 문자리터럴 연결하기 : format! 매크로 사용하기 

In [99]:
fn add_format(first:&str, second:&str) -> String {
    format!(" {} {}",first,second)
}


In [100]:
add_format("주주","하하")

" 주주 하하"

###  두 개의 문자리터럴 연결하기 : concat! 매크로 사용하기 

In [102]:
fn add_concat()->String {
    concat!("First", " ", "Second").to_string()
}

In [103]:
add_concat()

"First Second"

###  두 개의 문자리터럴 연결하기 : 문자열의 push_str 메서드 사용하기 

In [105]:
fn add_Str(first:&str, second:&str) -> String {
    let mut first = first.to_string();
    first.push_str(second);
    first
} 

In [106]:
add_Str("저저","야야")

"저저야야"

###  두 개의 문자리터럴 연결하기 : 문자열의 더하기 연산자  사용하기 

In [107]:
fn add_Add(first:&str, second:&str) -> String {
    let mut first = first.to_string();
    first + second
} 

In [108]:
add_Add("장하다", "문자열")

"장하다문자열"

###  Into 트레이트의 into 메서드로 문자열 결합하기

- 더하기 연산자를 사용할 때는 반드시 두번째는 as_str로 변환해서 처리

In [131]:
fn foo<T: Into<String>>(s: T, s1: T) -> String {
    s.into() + s1.into().as_str()
}

In [132]:
let s = String::from("abc");
let ss = String::from("def");

In [133]:
foo(s,ss)

"abcdef"

In [134]:
foo("abc","xyz")

"abcxyz"